In [20]:
# 현재 나의 작업 위치 확인
import os
os.getcwd()

'/home/ubuntu/Project/SJ'

In [1]:
import pandas as pd # pandas 라이브러리 pd로 불러옴
data = pd.read_excel('/home/ubuntu/Project/SJ/Data9.xlsx') # Data9 엑셀 파일을 불러서 data에 저장

In [29]:
data.head()

,발화자,발화문,QA여부,인텐트
0,c,"참치타다끼샐러드는 취소했었는데, 맞죠?",q,주문_취소_확인
1,s,"네, 참치타다끼샐러드는 취소된 상태입니다.",a,주문_취소_확인
2,c,주문한 메뉴 취소할 때 주문서만 있으면 된다는 말씀이세요?,q,주문_취소_확인
3,s,"네, 주문서만 있으면 취소 가능합니다.",a,주문_취소_확인
4,c,고객서비스가 엉망이군요. 그냥 갑니다.,q,주문_취소_확인


In [2]:
import torch # PyTorch 라이브러리 가져오기 / PyTorch : 딥러닝 모델 구축시켜 학습시키는데 사용되는 라이브러리
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast # Hugging face의 transformer라이브러리에서 GPT2LMHeadModel, PreTrainedTokenizerFast 모델을 불러옴
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2') # 사전에 학습된 skt kogpt2를 model에 불러옴
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>',
    unk_token='<unk>', pad_token='<pad>', mask_token='<mask>') # 사전에 학습된 skt kogpt2를 토크나이저 로드 

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
# input_ids : 토큰화된 텍스트를 나타내는 숫자 ID 시퀀스
# ex) '안녕하세요'  -> '안녕','하세요' -> 17,123 이런식으로 변한 숫자의 시퀀스가 input_ids
# 그럼 시퀀스는 뭐냐 -> 17, 123으로 변한 이런 숫자를 연속적으로 나열한거를 시퀀스라 한다
# 즉 순서대로 나열된 요소들의 연속체이다.

# attention_mask : 모델이 주의를 기울여야 할 토큰
# transformer는 모든 입력이 일정한 길이를 가져야한다 그래서 max_length를 정해놓고 남은 부분은 padding을 통해서 채우는데,
# 이렇게 패딩된 부분을 모델이 무시하게 해주는 역할
# ex) max_length = 5 , padding = max_length 일때 '안녕하세요' -> '안녕','하세요' -> 17, 123 -> [1, 1, 0, 0, 0]
# 여기서 1이 바로 주의를 기울여야할 토큰, 0은 무시하는 토큰

In [18]:
from torch.utils.data import Dataset # Dataset 로드
class ChatDataset(Dataset): # torch.utils.data.Dataset을 상속받는 ChatDataset클래스 생성
    def __init__(self, conversations, tokenizer, max_length=512): # 클래스 초기화 생성자, 대화데이터 conversations와 tokenizer, 최대길이를 입력받아 클래스 초기화
        self.tokenizer = tokenizer #전달받은 토크나이저 객체를 클래스의 인스턴스 변수로 저장
        self.inputs = []           #토큰화된 텍스트의 입력 ID들을 저장하는 리스트
        self.attention_masks = []  # attention_masks를 저장할 리스트

        for conv in conversations: # 반복문 대화에 의해 실행 
            # 각 대화 쌍의 질문과 답변을 결합하고, 각 부분 사이에 토크나이저의 끝경계 토큰을 삽입(더 잘 학습하기위해)
            combined_text = f"{conv['question']} {tokenizer.eos_token} {conv['answer']} {tokenizer.eos_token}"
            
            # 결합된 텍스트 conbined_text를 토크나이저를 사용해 토큰화/ 길이는 max_length만큼, max_length만큼 채워주게 padding
            tokenized_text = tokenizer(combined_text, truncation=True, max_length=max_length, padding="max_length")

            # 텍스트의 토큰화된 입력id들을 self.inputs리스트에 추가
            self.inputs.append(torch.tensor(tokenized_text['input_ids']))
            
            # 텍스트의 토큰화된 attention_mask들을 self.attenstion_masks 리스트에 추가
            self.attention_masks.append(torch.tensor(tokenized_text['attention_mask']))
    def __len__(self): # ChatDataset의 길이를 반환
        return len(self.inputs) # self.inputs 리스트의 길이를 반환 ( 데이터셋에 포함된 대화 쌍의 수)

    def __getitem__(self, idx): # 특정 인덱스인 idx에 해당하는 데이터 쌍을 반환 (데이터 쌍 : input_ids + attention_masks)
        return self.inputs[idx], self.attention_masks[idx]

In [19]:
# 데이터셋 변환
def build_dataset(data, tokenizer):    # 위에서 정의한 data와 tokenizer를 매개변수로 받아 새로운 데이터셋을 구축하는 함수
    conversations = []                 # conversations(대화) 라는 질문과 답변을 저장할 리스트 초기화
    for i in range(len(data) - 1):     # data의 길이 -1 만큼 for문을 돌릴건데, 왜 -1을 해야하느냐? -> 쌍을 지어야하는데 마지막 까지 가버리면 마지막이 A일때 쌍을 지을 수 없으니까!
        if data.iloc[i]['QA여부'] == 'q' and data.iloc[i+1]['QA여부'] == 'a': # for문을 돌면서 conversations에 질문과 답변을 쌍을 지어 추가
            conversations.append({
                'question': data.iloc[i]['발화문'],
                'answer': data.iloc[i+1]['발화문']
            })
    return conversations 



In [20]:
conversations = build_dataset(data, tokenizer) # 위에서 언급한 build_dataset함수를 사용해서 대화 생성

In [21]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# 전체 데이터셋을 학습 데이터셋과 검증 데이터셋으로 9:1로 분리
train_conversations, val_conversations = train_test_split(conversations, test_size=0.1)

# 학습 및 검증 데이터셋을 위한 DataLoader 생성 
train_dataset = ChatDataset(train_conversations, tokenizer) #위에서 언급했던 ChatDataset 클래스 활용해서 데이터셋 설정
val_dataset = ChatDataset(val_conversations, tokenizer)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # 위에서 불러온 DataLoader를 이용
validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) # 그럼 DataLoader는 뭐냐?

In [ ]:
# DataLoader는 PyTorch에서 제공하는 유틸리티 클래스
# 모델 학습을 위해 데이터를 효율적으로 로딩하고 배치처리를 수행하는 역할
# 그럼 여기서 배치는 뭐냐? 한번에 처리할 수 있는 일종의 묶음이라고 보면됨.
# 병렬로 묶어서 한번에 계산을 하는느낌인데 이때 너무 batch를 키우면 메모리가 터짐
# 셔플링도 가능 -> shuffle=True이면 각에포크가 시작할때 랜덤으로 데이터셋을 섞음

In [7]:
from transformers import AdamW
from tqdm import tqdm

# model.parameters의 파라미터는 매개변수를 의미, 즉 파라미터 = 매개변수 = 가중치, 편향
# 즉 모델의 파라미터를 최적화 하기 위해서 lr을 10^-5 x 5로 옵티마이저를 설정하는 과정
optimizer = AdamW(model.parameters(), lr=5e-5) 

# 학습 설정
epochs = 5 # 에폭설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # cuda(GPU)쓸거다 없으면 cpu쓸거다는 뜻
model.to(device) # 모델을 GPU로 돌릴거야!

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [8]:
for epoch in range(epochs): # 위에서 설정한 에폭만큼 epoch돌릴거야!
    model.train() #모델을 학습모드로 설정
    total_loss = 0 # 전체 로스 초기화
    for batch in tqdm(train_loader):   # 학습 데이터셋의 배치에 대해 반복
        # 배치 데이터를 GPU로 옮김 (필요한 경우)
        inputs, masks = [x.to(device) for x in batch] # inputs, masks 데이터 GPU로 이동시켜 실행 / PyTorch는 모두 같은 device에서 실행해야 오류가 안남

        # 그레이디언트 초기화
        optimizer.zero_grad() # 이전 그레디언트가 영향을 끼치지 않도록 초기화

        # 모델의 결과를 얻음
        outputs = model(input_ids=inputs, attention_mask=masks, labels=inputs) # 이부분은 아래에 설명
        loss = outputs.loss # 출력의 손실을 loss에 저장

        # 역전파 수행
        loss.backward() # 손실에 대한 그레디언트 계산해서 가중치를 어떻게 조절할지 결정
        optimizer.step() # 다시 옵티마이저를 통해 가중치를 조절하고

        total_loss += loss.item() # 총 손실을 업데이트해서 보여줌
    
    # 에포크마다 평균 손실을 출력
    print(f"Epoch {epoch + 1}/{epochs} completed. Average loss: {total_loss / len(train_loader)}")

100%|██████████| 3358/3358 [21:55<00:00,  2.55it/s]


Epoch 1/5 completed. Average loss: 0.10599597483238803


100%|██████████| 3358/3358 [21:55<00:00,  2.55it/s]


Epoch 2/5 completed. Average loss: 0.0765620187100216


100%|██████████| 3358/3358 [21:56<00:00,  2.55it/s]


Epoch 3/5 completed. Average loss: 0.06841152783831611


100%|██████████| 3358/3358 [23:31<00:00,  2.38it/s]


Epoch 4/5 completed. Average loss: 0.06126628282628044


100%|██████████| 3358/3358 [47:19<00:00,  1.18it/s]

Epoch 5/5 completed. Average loss: 0.055123316653254775


In [ ]:
# model(input_ids=inputs, attention_mask=masks, labels=inputs)
# model은 우리가 위에서 설정햇듯 sktkogpt2이고
# input_ids역시 위에서 설명햇듯이 입력된 텍스트를 숫자로 저장한 시퀀스
# inputs은 batch를 실행한 입력 데이터를 의미
# attention_mask 역시 위에서 설명했듯 어디에 집중을 할지
# mask는 batch의 attention_mask를 나타냄
# labels = inputs은 labels에 inputs을 전달하는건데 일종의 자기주도학습으로 입력을 정확하게 재현하기 위해 설정

In [15]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>', eos_token='</s>',
    unk_token='<unk>', pad_token='<pad>', mask_token='<mask>',
    padding_side='left'
) # 이건 위에서 한 것과 같은데 패딩만 오른쪽으로 하다가 왼쪽으로 하는 코드만 추가 / [1,1,0,0,0] -> [0,0,0,1,1]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [22]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
import numpy as np

# BLEU 점수 계산을 위한 함수
def calculate_bleu(reference_texts, generated_text):
    # 참조 텍스트를 토큰화
    reference_tokens = [word_tokenize(ref) for ref in reference_texts]
    # 생성된 텍스트를 토큰화
    generated_tokens = word_tokenize(generated_text)

    smoothing = SmoothingFunction().method1 # 0으로 나눠버리는 등의 문제를 방지하기위해 Smoothing함수(method1) 사용
    return sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smoothing)
    # sentence_bleu 이건 NLTK 라이브러리 함수인데 ref, gen 텍스트 간의 BLEU점수 계

In [23]:
# 검증 데이터셋에 대한 BLEU 점수 계산
model.eval() # 모델을 평가모드로 설정
total_eval_loss = 0 # 총 손실을 초기화
bleu_scores = [] # bleu_scores를 넣을 리스트 초기화

for batch in tqdm(validation_loader):
    inputs, masks = [x.to(device) for x in batch]

    with torch.no_grad(): # PyTorch 컨텍스트 매니저 / 이 블록에서 수행하는 연산은 그레디언트 추적 x / 이유는 메모리 절약, 계산효율성 증가, 모델 가중치 업데이트 방지
        # forward 메서드를 사용하여 손실 계산
        outputs = model(input_ids=inputs, attention_mask=masks, labels=inputs)
        loss = outputs.loss
        total_eval_loss += loss.item()

        # 모델의 출력 생성 (생성된 텍스트의 품질 평가용)
        generated_outputs = model.generate(input_ids=inputs, max_length=100) # 모델을 활용해 새로운 텍스트 생성 / 길이는 100

    # BLEU 점수 계산
    for i in range(inputs.size(0)): # inputs.size(0)는 텐서의 첫번째 차원의 크기 / 텐서의 첫번째 차원의 크기 = 배치 크기
        # decode는 encode한걸 다시 텍스트로 돌리기
        ref_text = tokenizer.decode(inputs[i], skip_special_tokens=True) # inputs[i] : 현재 샘플의 입력 데이터, skip_special_tokens : 특수 토큰들 제외
        gen_text = tokenizer.decode(generated_outputs[i], skip_special_tokens=True)
        bleu_score = calculate_bleu([ref_text], gen_text)
        bleu_scores.append(bleu_score)

# 평균 손실 및 BLEU 점수 출력
avg_val_loss = total_eval_loss / len(validation_loader)
avg_bleu_score = np.mean(bleu_scores)
print(f"Validation Loss: {avg_val_loss}")
print(f"Average BLEU Score: {avg_bleu_score}")

100%|██████████| 374/374 [01:46<00:00,  3.50it/s]

Validation Loss: 3.3711369394618558
Average BLEU Score: 0.9790509127219046


In [9]:
# model.eval()
# total_eval_loss = 0
# for batch in tqdm(validation_loader):  # 검증 데이터 로더
#     inputs, masks = [x.to(device) for x in batch]

#     with torch.no_grad():
#         outputs = model(input_ids=inputs, attention_mask=masks, labels=inputs)
#         loss = outputs.loss
#         total_eval_loss += loss.item()

 11%|█         | 41/374 [00:12<01:45,  3.17it/s]


KeyboardInterrupt: 

In [24]:
# # 평균 검증 손실 계산
# avg_val_loss = total_eval_loss / len(validation_loader)
# print(f"Validation Loss: {avg_val_loss}")

Validation Loss: 3.3711369394618558


In [25]:
# def generate_response(sentence, model, tokenizer, device):
#     # 입력 문장 토큰화
#     input_ids = tokenizer.encode(sentence + tokenizer.eos_token, return_tensors='pt')
#     input_ids = input_ids.to(device)

#     # 모델이 응답 생성
#     with torch.no_grad():
#         output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1)
    
#     # 생성된 토큰을 문자열로 변환
#     full_response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

#     # 입력 문장 이후의 부분만 반환
#     # 가정: 응답은 '입력 문장 응답' 형식으로 구성됨
#     response_start = full_response.find(sentence) + len(sentence)
#     if response_start != -1:
#         return full_response[response_start:].strip()
#     else:
#         return full_response

In [27]:
# 예시 문장으로 응답 생성
while True:
    sentence = input()
    response = generate_response(sentence, model, tokenizer, device)
    print(response)
    if sentence == '수고하세요':
        break

 그냥 배달 수수료 있나요


네


 얼마예요


만삼천원입니다


 배달 시키면 배달 비 있어요?


...

네


 통순살흑마늘간장치킨 배달하면 배달비 얼마에요?


2000원이요


 수고하세요


네


In [28]:
# 모델 저장
model_path = "/home/ubuntu/Project/SJ/saved_model"
# save_pretrained(model_path) : PyTorch에서 huggingface의 transformers 라이브러리를 사용할 때 모델을 저장하는 방법
# 모델의 가중치와 설정을 지정된 경로에 저장함
model.save_pretrained(model_path)  
tokenizer.save_pretrained(model_path)

('/home/ubuntu/Project/SJ/saved_model/tokenizer_config.json',
 '/home/ubuntu/Project/SJ/saved_model/special_tokens_map.json',
 '/home/ubuntu/Project/SJ/saved_model/tokenizer.json')